In [1]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [3]:

%pip install matplotlib
%pip install scikit-learn
%pip install tensorflow
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


import and setup

In [5]:
%pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [2]:

import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
from tqdm import tqdm

#paths
IMAGE_DIR = 'C:/Users\PartZ/derm-ai-assistant/data/ISIC_2018/images/ISIC2018_Task1-2_Training_Input'
MASK_DIR = 'C:/Users/PartZ/derm-ai-assistant/data/ISIC_2018/masks/ISIC2018_Task1_Training_GroundTruth'

#settings
IMG_SIZE= 128

load and resize images and masks

In [3]:
def load_data(IMAGE_DIR, MASK_DIR, IMG_SIZE=128):
    images = []
    masks = []

    img_names = os.listdir(IMAGE_DIR)
    img_names = [f for f in img_names if f.endswith('.jpg')]

    for name in tqdm(img_names[:1000]):
        img_path = os.path.join(IMAGE_DIR, name)
        mask_path = os.path.join(MASK_DIR, name.replace('.jpg','_segmentation.png'))


        img = cv2.imread(img_path)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img / 255.0

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, (IMG_SIZE,IMG_SIZE))
        mask = mask / 255.0
        mask = np.expand_dims(mask, axis=-1)

        images.append(img)
        masks.append(mask)
    
    return np.array(images), np.array(masks)

images, masks = load_data(IMAGE_DIR, MASK_DIR, IMG_SIZE)
print("loaded images:", images.shape)
print("loaded masks:", masks.shape)

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:46<00:00, 21.59it/s]


loaded images: (1000, 128, 128, 3)
loaded masks: (1000, 128, 128, 1)


train/test split

In [4]:
x_train, x_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)

Unet

In [6]:
def conv_block(input, num_filters):
    x= layers.Conv2D(num_filters, 3, padding="same", activation="relu")(input)
    x = layers.Conv2D(num_filters, 3, padding="same", activation="relu")(x)
    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = layers.MaxPooling2D((2,2))(x)
    return x,p

def decoder_block(input, skip, num_filters):
    x = layers.Conv2DTranspose(num_filters, (2,2), strides=2, padding="same")(input)
    x = layers.concatenate([x, skip])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = layers.Input(shape=input_shape)

    #encoder
    s1, p1 = encoder_block(inputs, 64)
    s2,p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2,256)

    #bottleneck
    b = conv_block(p3, 512)

    #decoder
    d3 = decoder_block(b, s3, 512)
    d2 = decoder_block(d3 , s2, 128)
    d1 = decoder_block(d2, s1, 64)

    outputs = layers.Conv2D(1,(1,1),padding="same", activation="sigmoid")(d1)

    return models.Model(inputs, outputs)

model = build_unet((IMG_SIZE,IMG_SIZE,3))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 128, 128,  │      1,792 │ input_layer_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 128, 128,  │     36,928 │ conv2d_8[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 64, 64,    │          0 │ conv2d_9[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 64, 64,    │     73,856 │ max_pooling2d_3[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 64, 64,    │    147,584 │ conv2d_10[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 32, 32,    │          0 │ conv2d_11[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 32, 32,    │    295,168 │ max_pooling2d_4[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_13 (Conv2D)  │ (None, 32, 32,    │    590,080 │ conv2d_12[0][0]   │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_5     │ (None, 16, 16,    │          0 │ conv2d_13[0][0]   │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 16, 16,    │  1,180,160 │ max_pooling2d_5[… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_15 (Conv2D)  │ (None, 16, 16,    │  2,359,808 │ conv2d_14[0][0]   │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_1  │ (None, 32, 32,    │  1,049,088 │ conv2d_15[0][0]   │
│ (Conv2DTranspose)   │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 32, 32,    │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 768)              │            │ conv2d_13[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_16 (Conv2D)  │ (None, 32, 32,    │  3,539,456 │ concatenate[0][0] │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_17 (Conv2D)  │ (None, 32, 32,    │  2,359,808 │ conv2d_16[0][0]   │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_2  │ (None, 64, 64,    │    262,272 │ conv2d_17[0][0] 

 Total params: 12,482,241 (47.62 MB)

 Trainable params: 12,482,241 (47.62 MB)

 Non-trainable params: 0 (0.00 B)

Train

In [7]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs = 10,
    batch_size = 16
)

Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 304s 6s/step - accuracy: 0.7001 - loss: 0.5843 - val_accuracy: 0.8327 - val_loss: 0.4246
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 294s 6s/step - accuracy: 0.8174 - loss: 0.4621 - val_accuracy: 0.8444 - val_loss: 0.4063
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 305s 6s/step - accuracy: 0.8467 - loss: 0.3973 - val_accuracy: 0.8508 - val_loss: 0.3731
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 304s 6s/step - accuracy: 0.8443 - loss: 0.3902 - val_accuracy: 0.8463 - val_loss: 0.3842
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 303s 6s/step - accuracy: 0.8385 - loss: 0.4098 - val_accuracy: 0.8147 - val_loss: 0.4543
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 303s 6s/step - accuracy: 0.8201 - loss: 0.4528 - val_accuracy: 0.8412 - val_loss: 0.3808
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 410s 8s/step - accuracy: 0.8521 - loss: 0.3680 - val_accuracy: 0.8500 - val_loss: 0.3714
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 530s 11s/step - accuracy: 0.8579 - loss: 0.3547 - val_accuracy: 0.8534 - 

evaluate and save

In [10]:
model.save("C:/Users/PartZ/derm-ai-assistant/backend/model/skin_segmentation_unet.keras")